In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

df = pd.read_csv('real_estate_rental.csv')

1. Filter by Multiple Conditions

Find all properties in Makati that have a monthly rent greater than ₱50,000 and are Fully Furnished. Display the property name, rent, and floor area.

In [3]:


makati_full_furnish = df[(df['location'] == 'Makati') & (df['furnishing']== 'Fully Furnished') & (df['monthly_rent'] > 50000)]
makati_full_furnish[['property_name','monthly_rent','furnishing','location']]



,property_name,monthly_rent,furnishing,location
5,Skyline Condo 22F,75000,Fully Furnished,Makati
8,Executive Suites 10A,55000,Fully Furnished,Makati
12,Palm Grove Condo 18E,80000,Fully Furnished,Makati
16,Luxury Penthouse 30A,250000,Fully Furnished,Makati
23,Tower One 25G,60000,Fully Furnished,Makati


2. Calculate Rent per Square Meter

Add a new column rent_per_sqm that calculates the monthly rent divided by floor area. Which property has the highest and lowest rent per square meter?

In [4]:
df['rent_per_sqft'] = df['monthly_rent'] / df['floor_area_sqm']

highest_rent_per_sqft = df.nlargest(1,'rent_per_sqft')
highest_rent_per_sqft[[

    'property_name',
    'monthly_rent',
    'floor_area_sqm',
    'rent_per_sqft'
]]

,property_name,monthly_rent,floor_area_sqm,rent_per_sqft
16,Luxury Penthouse 30A,250000,200,1250.0


3. Identify Overdue Payments

Create a summary showing all properties with Overdue payment status. Include the property name, tenant name, location, and how many months they've been renting (assume current date is January 2025).

In [5]:
df['tenant_move_in_date'] = pd.to_datetime(df['tenant_move_in_date'])

overdue_payment = df[df['payment_status'] == 'Overdue']


overdue_payment = (
    df['tenant_move_in_date']
    .dt.to_period('Q')
    .value_counts()
    .sort_index()
)


overdue_payment




tenant_move_in_date
2024Q1     5
2024Q2    11
2024Q3     9
2024Q4     9
2025Q1     1
Freq: Q-DEC, Name: count, dtype: int64

In [6]:
group_location = df.groupby('location').agg(
    {'property_name':'count',
    'monthly_rent':['mean','max','min'],

    })

group_location

property_name   monthly_rent                
                    count           mean     max     min
location                                                
Alabang                 1  180000.000000  180000  180000
Cavite                  1   65000.000000   65000   65000
Las Pinas               1  120000.000000  120000  120000
Makati                  7   83857.142857  250000   32000
Mandaluyong             4   36250.000000   40000   32000
Manila                  2   20000.000000   28000   12000
Muntinlupa              1  130000.000000  130000  130000
Paranaque               2   61500.000000   68000   55000
Pasay                   3   40000.000000   50000   25000
Pasig                   4   37500.000000   48000   15000
Quezon City             4   70000.000000   95000   48000
Taguig                  5   49600.000000   95000   18000

Agent Performance Analysis

In [7]:
df['total_commission'] = df['monthly_rent'] * df['commission_rate']

agent_performance = df.groupby('agent_name').agg({
    'property_name':'count',
    'total_commission':['sum','mean'],
 
})

agent_performance




property_name total_commission             
                     count              sum         mean
agent_name                                              
Carlos Tan              11          44350.0  4031.818182
Jose Garcia             11          20670.0  1879.090909
Maria Santos            13          66110.0  5085.384615

Property Type Comparison

Compare different property_type categories by:

Average bedrooms and bathrooms
Average floor area
Average monthly rent
Count of properties

In [8]:
property_type_comparison = df.groupby('property_type').agg(
    average_rent=('monthly_rent', 'mean'),
    average_bedrooms=('bedrooms', 'mean'),
    property_count=('property_name', 'count')
)

property_type_comparison


,average_rent,average_bedrooms,property_count
property_type,,,
Apartment,36166.666667,2.333333,6
Condominium,46150.000000,1.900000,20
House,112500.000000,4.666667,6
Penthouse,250000.000000,4.000000,1
Townhouse,61500.000000,3.000000,2


Advanced Analysis
7. Furnishing Impact on Rent

Analyze how furnishing status affects rent. Calculate the average rent per sqm for each furnishing type (Fully Furnished, Semi-Furnished, Unfurnished). Which furnishing type commands the highest premium?

In [14]:
furnishing_impact = (
    df.groupby('furnishing').agg(

        avg_rent_per_sqm=('rent_per_sqft','mean'),
        property_count=('rent_per_sqft','count')

    )
)

furnishing_impact


,avg_rent_per_sqm,property_count
furnishing,,
Fully Furnished,821.163065,19
Semi-Furnished,609.824822,10
Unfurnished,464.814815,6


In [16]:
lease_term_analysis = df.groupby('lease_term_months').agg(
    avg_rent_per_sqm=('rent_per_sqft','mean'),
    avg_property_age=('property_age_years','mean'),
    avg_security_deposit=('security_deposit','mean'),
    property_count=('property_id','count')
).reset_index().sort_values(by='lease_term_months',ascending=False)


lease_term_analysis






,lease_term_months,avg_rent_per_sqm,avg_property_age,avg_security_deposit,property_count
2,24,463.869048,10.125000,199500.0,8
1,12,791.340932,5.863636,118000.0,22
0,6,673.756494,9.600000,36800.0,5


In [ ]:

# create amenities features
df['has_pool'] = df['ammeties'].str.contains('Pool',case=False,na=False)
df['has_parking'] = df['ammeties'].str.contains('Parking',case=False,na=False)


pool_comparison = (
    df.groupby('has_pool')['monthly_rent'].mean().rename({True:'With Pool',False:'Without Pool'})
)


